# 1) Data Collection - Spotify 200 Charts Analysis

The following Notebook is part 1 to a four-part series. This Notebook, specifically, collects data from [kworb.net](https://kworb.net/spotify/), which itself is sourced from [Spotify](https://charts.spotify.com/charts/overview/us). In this sheet, we do the following:

- Collect all URLs from kworb.net's USA Chart
- Aggregate data from all URls collected (each URL has different column lengths) into one dataframe
- Clean the data to a dataset of only 3 columns
    - Date
    - Streams
    - Title

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
from tqdm import tqdm
import random
import csv

# Obtain all URLs from the [Kworb USA site](https://kworb.net/spotify/country/us_weekly_totals.html)




In [2]:
parent_url = 'https://kworb.net/spotify/country/us_weekly_totals.html' # get kworb us weekly totals url

# get the page via BeautifulSoup
page = requests.get(parent_url)
soup = BeautifulSoup(page.content, 'html.parser')

# get the table
table = soup.find('table')
rows = table.find_all('tr')


def get_track_href(row):
    # Find the first 'a' tag in the row where the 'href' contains the word 'track'
    track_link = row.find('a', href=lambda href: href and 'track' in href)
    
    # If such a link is found, extract and return the 'href' attribute
    if track_link:
        track_href = track_link['href']
        return track_href
    
    # If no track link is found, return None
    return None

song_url = 'https://kworb.net/spotify'

# Read the CSV into a DataFrame
links_file_path = '/Users/jeremiasfigueroa/Desktop/Data Projects/USA Hit Music Analysis Project/Spotify 200 Analysis/updated_kworb_links.csv'

# Open file in write mode to clear previous content
with open(links_file_path, 'w') as f:
    pass  # Just to clear the file

try:
    df = pd.read_csv(links_file_path)
    if df.empty:
        print("The file is empty and the DataFrame has no data.")
    else:
        print("The DataFrame has data.")
except pd.errors.EmptyDataError:
    print("The kworb links file is empty. You may proceed to add URLs.")

The kworb links file is empty. You may proceed to add URLs.


In [3]:
# Now append the new URLs
for row in rows:
    try:
        track_href = get_track_href(row)
        if track_href:
            # remove the first two '..' in the href
            track_href = track_href[2:]
            # save it to csv file
            with open(links_file_path, 'a') as f:
                f.write(song_url + track_href + '\n')
    except:
        pass

# Load the CSV file to verify the URLs
song_urls = pd.read_csv(links_file_path, header=None)
song_urls

,0
0,https://kworb.net/spotify/track/0RiRZpuVRbi7oq...
1,https://kworb.net/spotify/track/68Dni7IE4VyPkT...
2,https://kworb.net/spotify/track/285pBltuF7vW8T...
3,https://kworb.net/spotify/track/2QjOHCTQ1Jl3za...
4,https://kworb.net/spotify/track/7GX5flRQZVHRAG...
...,...
8682,https://kworb.net/spotify/track/0jhYMcAqY9LkMC...
8683,https://kworb.net/spotify/track/2DOykLfhy5XEHx...
8684,https://kworb.net/spotify/track/72Yfw4fVEbJNGa...
8685,https://kworb.net/spotify/track/7KKW3MSfqCCai7...


# Retrieve raw data from each link above and write into csv

First, create a date variable (to be used when naming CSV files)

In [4]:
from datetime import datetime

# Specify a custom date or leave it as None for today's date
custom_date = None  # Change this to a specific date if needed

# Use the custom date if specified; otherwise, use the current date
if custom_date:
    # Convert the string to a datetime object
    custom_date_obj = datetime.strptime(custom_date, '%m/%d/%Y')
    formatted_date = custom_date_obj.strftime('%B_%d_%Y').lower().replace(' ', '_')
else:
    current_date = datetime.now()
    formatted_date = current_date.strftime('%B_%d_%Y').lower().replace(' ', '_')


# Define the output CSV file with the dynamic date
output_csv = f'/Users/jeremiasfigueroa/Desktop/Data Projects/USA Hit Music Analysis Project/Spotify 200 Analysis/updated_kworb_data_{formatted_date}.csv'


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import random

# List of user agents to simulate different browsers
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/113.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:88.0) Gecko/20100101 Firefox/88.0",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1"
]

# Track processed URLs and any errors
processed_count = 0
error_urls = []

def process_spotify_chart(url, output_csv):
    global processed_count  # Access the counter defined outside the function
    headers = {
        'User-Agent': random.choice(USER_AGENTS)
    }
    
    # Fetch the HTML content from the URL
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        error_urls.append((url, f"Status code: {response.status_code}"))
        return
    
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.title.string.strip() if soup.title else "Unknown Title"

    # Find the table and check if it exists
    table = soup.find('table')
    if table is None:
        error_urls.append((url, "No table found"))
        return

    rows = table.find_all('tr')
    header = [th.text.strip() for th in rows[0].find_all('th')]

    # Extract data rows
    data = []
    for row in rows[1:]:
        cells = row.find_all('td')
        if cells:
            data.append([cell.text.strip() for cell in cells])

    df = pd.DataFrame(data, columns=header)
    df['Title'] = title
    df.to_csv(output_csv, mode='a', index=False, header=True)

    processed_count += 1  # Increment the counter

    # Print only every 100 processed URLs
    if processed_count % 100 == 0:
        print(f"{processed_count} URLs processed...")

# Load URLs
links_df = pd.read_csv('/Users/jeremiasfigueroa/Desktop/Data Projects/USA Hit Music Analysis Project/Spotify 200 Analysis/updated_kworb_links.csv', header=None)
links_df.columns = ['url']

# Process URLs
for i, url in enumerate(tqdm(links_df['url'], desc="Processing URLs"), start=1):
    process_spotify_chart(url, output_csv)

# Final summary
print(f"\nProcessing completed. Total URLs processed successfully: {processed_count}")
if error_urls:
    print(f"Total URLs with errors: {len(error_urls)}")
    for url, error in error_urls:
        print(f"- {url}: {error}")



Processing URLs:   1%|          | 100/8687 [02:30<2:57:57,  1.24s/it]

100 URLs processed...


Processing URLs:   2%|▏         | 200/8687 [04:48<3:29:12,  1.48s/it]

200 URLs processed...


Processing URLs:   3%|▎         | 300/8687 [07:05<3:10:40,  1.36s/it]

...

8500 URLs processed...


Processing URLs:  99%|█████████▉| 8600/8687 [1:50:51<00:46,  1.85it/s]

8600 URLs processed...


Processing URLs: 100%|██████████| 8687/8687 [1:51:39<00:00,  1.30it/s]


Processing completed. Total URLs processed successfully: 8687


# Clean the csv
The csv is a collection of smaller csv's stacked on top of each other, and there is an inconsistent number of columns for each smaller csv. 

Therefore, we need to determine the max number of columns before importing the csv.


In [6]:
csv_to_open = output_csv #f'/Users/jeremiasfigueroa/Desktop/Data Projects/USA Hit Music Analysis Project/Spotify 200 Analysis/updated_kworb_data_{formatted_date}.csv'

# Step 1: Read the file and determine the maximum number of columns
with open(csv_to_open, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    max_cols = max(len(row) for row in reader)

# Step 2: Re-read the file with the maximum number of columns
df = pd.read_csv(
    csv_to_open,
    engine='python', header=None, on_bad_lines='skip', names=range(max_cols)
)

df

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,Date,Global,US,GB,AU,CA,MX,PH,BR,IN,...,KZ,VE,MA,PK,UY,UA,Title,None,None,None
1,Total,"3,478,314,272","1,172,234,001","179,080,240","171,047,349","158,845,706","122,047,420","86,631,462","52,750,370","42,557,037",...,"205,312","164,680","139,787","139,773","51,103","37,634",Post Malone - Sunflower - Spider-Man: Into the...,None,None,None
2,Peak,"1 (34,579,416)","1 (14,731,867)","3 (2,334,687)","1 (2,088,250)","2 (1,749,717)","8 (1,998,351)","3 (1,649,280)","56 (1,128,745)","3 (1,227,154)",...,"50 (32,781)","102 (30,969)","68 (7,326)","157 (29,363)","151 (11,320)","155 (34,910)",Post Malone - Sunflower - Spider-Man: Into the...,None,None,None
3,2018/10/25,"5 (25,233,843)","2 (10,667,603)","10 (1,938,027)","11 (1,092,314)","2 (1,580,585)","71 (613,940)","43 (424,033)","135 (573,081)",--,...,--,--,--,--,--,--,Post Malone - Sunflower - Spider-Man: Into the...,None,None,None
4,2018/11/01,"5 (25,456,872)","4 (10,215,607)","8 (1,956,212)","2 (1,693,971)","2 (1,496,338)","49 (830,044)","31 (597,032)","121 (606,868)",--,...,--,--,--,--,--,--,Post Malone - Sunflower - Spider-Man: Into the...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299397,2015/03/22,"253 (68,574)",Earl Sweatshirt - Faucet - Spotify Chart History,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299398,Date,US,Title,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299399,Total,"67,174",Big Sean - Blessings - Extended Version - Spot...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
299400,Peak,"248 (67,174)",Big Sean - Blessings - Extended Version - Spot...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [7]:
# Initialize variables
extracted_data = []
current_chunk = None

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Identify the start of a new mini-CSV by checking for the 'Date', 'Title', and 'US' columns in a row
    if row.str.contains('Date', case=False).any() and row.str.contains('Title', case=False).any() and row.str.contains('US', case=False).any():
        # Process the previous chunk if it exists
        if current_chunk is not None:
            # Extract the 'Date', 'US', and 'Title' columns if they exist in the current chunk
            try:
                extracted_data.append(current_chunk[['Date', 'US', 'Title']])
            except KeyError:
                pass  # In case one of these columns is missing, skip this chunk

        # Start a new chunk with current row as the header
        current_chunk = pd.DataFrame(columns=row)
        
    # If we're in the middle of processing a chunk, append rows to the current chunk
    elif current_chunk is not None:
        # Convert row to DataFrame and add it to current_chunk using concat
        row_df = pd.DataFrame([row.values], columns=current_chunk.columns)
        current_chunk = pd.concat([current_chunk, row_df], ignore_index=True)

# Process the last chunk if it exists
if current_chunk is not None:
    try:
        extracted_data.append(current_chunk[['Date', 'US', 'Title']])
    except KeyError:
        pass

# Concatenate all chunks into a final DataFrame
if extracted_data:
    final_df = pd.concat(extracted_data, ignore_index=True)
else:
    final_df = pd.DataFrame(columns=['Date', 'US', 'Title'])  # Empty DataFrame in case of no data

final_csv = f'/Users/jeremiasfigueroa/Desktop/Data Projects/USA Hit Music Analysis Project/Spotify 200 Analysis/CLEANED_spotify_chart_data_{formatted_date}.csv'

# Save the result to a new CSV file
final_df.to_csv(final_csv, index=False)

# Check how many songs are present
print(len(final_df[(final_df['Date']=='Total')]))

# # Check the result
final_df

8687


,Date,US,Title
0,Total,"1,172,234,001",Post Malone - Sunflower - Spider-Man: Into the...
1,Peak,"1 (14,731,867)",Post Malone - Sunflower - Spider-Man: Into the...
2,2018/10/25,"2 (10,667,603)",Post Malone - Sunflower - Spider-Man: Into the...
3,2018/11/01,"4 (10,215,607)",Post Malone - Sunflower - Spider-Man: Into the...
4,2018/11/08,"5 (9,376,246)",Post Malone - Sunflower - Spider-Man: Into the...
...,...,...,...
290710,Peak,"253 (68,574)",Earl Sweatshirt - Faucet - Spotify Chart History
290711,2015/03/22,"253 (68,574)",Earl Sweatshirt - Faucet - Spotify Chart History
290712,Total,"67,174",Big Sean - Blessings - Extended Version - Spot...
290713,Peak,"248 (67,174)",Big Sean - Blessings - Extended Version - Spot...
